In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
filenames = glob.glob('Datathon*.csv')

In [3]:
pre_dat = pd.DataFrame({})
for ind, filename in enumerate(filenames):
    print(ind)
    tmp_dat = pd.read_csv(filename)
    pre_dat = pre_dat.append(tmp_dat)    
pre_dat.columns = ['BET_ID', 'BET_TRANS_ID', 'MATCH_BET_ID', 'ACCOUNT_ID',
       'COUNTRY_OF_RESIDENCE_NAME', 'PARENT_EVENT_ID', 'EVENT_ID', 'MATCH',
       'EVENT_NAME', 'EVENT_DT', 'OFF_DT', 'BID_TYP', 'STATUS_ID',
       'PLACED_DATE', 'TAKEN_DATE', 'SETTLED_DATE', 'CANCELLED_DATE',
       'SELECTION_NAME', 'PERSISTENCE_TYPE', 'BET_PRICE', 'PRICE_TAKEN',
       'INPLAY_BET', 'BET_SIZE', 'PROFIT_LOSS']
peek_dat = pd.read_csv('DSMDatathon_samplefile.txt', sep='\t')
pre_dat = peek_dat.append(pre_dat)

0
1

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



2
3
4


In [4]:
pre_dat.loc[:, 'BID_TYP'] = pre_dat.BID_TYP.str.strip(' ')
pre_dat.loc[:, 'STATUS_ID'] = pre_dat.STATUS_ID.str.strip(' ')

In [5]:
pre_dat = pre_dat.loc[(pre_dat.STATUS_ID == 'S').values]

In [20]:
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'correct_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'correct_profit_loss'] = - pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'correct_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'correct_profit_loss'] = - pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [6]:
# get penalised profit
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'penalty_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'penalty_profit_loss'] = - 2 * pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'penalty_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'penalty_profit_loss'] = - 2 * pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [5]:
pre_dat = pre_dat.fillna(0)

In [7]:
submit = pd.read_csv('sample_submission_bet_size.csv')
submit.columns = ['ACCOUNT_ID', 'Prediction']

In [30]:
fianl_dat = pd.read_csv('semi_and_final_features.csv')
fianl_dat.columns = ['Account_ID', 'EVENT_ID', 'TRANSACTION_COUNT', 'STATUS_ID',
       'INPLAY_BET', 'AVG_BET_SIZE', 'MAX_BET_SIZE', 'MIN_BET_SIZE',
       'STDEV_BET_SIZE']

In [31]:
fianl_dat = fianl_dat.loc[(fianl_dat.STATUS_ID == 'S').values]

In [32]:
fianl_dat.loc[:, 'total'] = fianl_dat.TRANSACTION_COUNT * fianl_dat.AVG_BET_SIZE

In [33]:
fianl_dat_total = fianl_dat.groupby('Account_ID')['total'].sum().loc[submit.ACCOUNT_ID.tolist()]

In [38]:
fianl_dat_total.head()

Account_ID
1004240     19027.790500
1009742    530423.584971
1003908      2264.014397
1020888       657.100800
1010074      2709.854433
Name: total, dtype: float64

In [8]:
pre_dat = pre_dat.loc[(pre_dat.ACCOUNT_ID.isin(submit.ACCOUNT_ID)).values]

In [9]:
pre_dat_by_account = pre_dat.groupby('ACCOUNT_ID')

In [10]:
penalty_profit = pre_dat_by_account['penalty_profit_loss'].sum()

In [16]:
penalty_profit.describe()

count        7374.000000
mean       -69340.751663
std        829091.546451
min     -33986767.260416
25%        -11804.534601
50%         -2919.291933
75%          -648.488988
max         90922.409558
Name: penalty_profit_loss, dtype: float64

In [12]:
penalty_profit = penalty_profit.loc[submit.ACCOUNT_ID.tolist()]

In [15]:
penalty_profit = penalty_profit.fillna(penalty_profit.median())

In [17]:
penalty_profit = penalty_profit.reset_index()

In [18]:
penalty_profit.columns = ['Account_ID', 'Prediction']

In [19]:
penalty_profit.to_csv('penalised_profit_meidan.csv', index=False)

In [9]:
pre_dat_by_account = pre_dat.groupby('ACCOUNT_ID')
prefit_rate = pre_dat_by_account['correct_profit_loss'].sum() / pre_dat_by_account['BET_SIZE'].sum()
prefit_rate = prefit_rate.loc[submit.ACCOUNT_ID.tolist()]
prefit_rate = prefit_rate.fillna(prefit_rate.min())
prefit_rate = prefit_rate.reset_index()
prefit_rate.columns = submit.columns

In [39]:
pre_dat_by_account = pre_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])
prefit_rate = pre_dat_by_account['correct_profit_loss'].sum() / pre_dat_by_account['BET_SIZE'].sum()
prefit_rate_avg = prefit_rate.groupby(level=0).mean()
prefit_rate_avg = prefit_rate_avg.loc[submit.ACCOUNT_ID.tolist()]
prefit_rate_avg = prefit_rate_avg.fillna(prefit_rate.mean())
pred_profit = prefit_rate_avg * fianl_dat_total
pred_profit = pred_profit.reset_index()
pred_profit.columns = ['Account_ID', 'Prediction']
pred_profit.to_csv('pred_profit_mean.csv', index=False)

In [40]:
pred_profit.head()

,Account_ID,Prediction
0,1004240,82.901261
1,1009742,11083.592476
2,1003908,1457.474331
3,1020888,-308.837376
4,1010074,-379.379621


In [24]:
prefit_rate_avg.describe()

,Account_ID,Prediction
count,7374.000000,7374.000000
mean,1011060.910768,-0.895287
std,6613.263866,2.198621
min,1000002.000000,-6.400000
25%,1005685.750000,-0.203869
50%,1010105.000000,-0.025812
75%,1016848.750000,0.035706
max,1024189.000000,3.550000


In [11]:
prefit_rate.head()

ACCOUNT_ID
1000002    0.020570
1000004   -0.128825
1000005   -0.014578
1000010   -0.021432
1000014    0.500000
dtype: float64

In [14]:
prefit_rate.shape

(7374,)

In [16]:
prefit_rate.name = 'Prediction'

In [20]:
prefit_rate.head()

,ACCOUNT_ID,Prediction
0,1004240,-0.005224
1,1009742,0.005226
2,1003908,0.003733
3,1020888,-0.470000
4,1010074,-0.140000


In [21]:
prefit_rate.to_csv('profit_rate_with_cancel.csv', index=False)

In [26]:
four_team_pre_dat = pre_dat.loc[(pre_dat.MATCH.str.contains('Australia') 
                                 | pre_dat.MATCH.str.contains('New Zealand') 
                                 | pre_dat.MATCH.str.contains('India')
                                 |pre_dat.MATCH.str.contains('South Africa')).values]

In [42]:
four_team_pre_dat_by_account = four_team_pre_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])
four_team_prefit_rate = four_team_pre_dat_by_account['correct_profit_loss'].sum() / four_team_pre_dat_by_account['BET_SIZE'].sum()
four_team_prefit_rate_avg = four_team_prefit_rate.groupby(level=0).mean()
four_team_prefit_rate_avg = four_team_prefit_rate_avg.loc[submit.ACCOUNT_ID.tolist()]
four_team_prefit_rate_avg = four_team_prefit_rate_avg.fillna(prefit_rate.mean())
four_team_pred_profit = four_team_prefit_rate_avg * fianl_dat_total
four_team_pred_profit = four_team_pred_profit.reset_index()
four_team_pred_profit.columns = ['Account_ID', 'Prediction']
four_team_pred_profit.to_csv('four_team_pred_profit_mean.csv', index=False)

In [29]:
four_team_prefit_rate_avg.head()

,Account_ID,Prediction
0,1004240,-0.032147
1,1009742,0.036637
2,1003908,0.662868
3,1020888,-0.470000
4,1010074,-0.140000


In [36]:
four_team_pre_dat.head()

,BET_ID,BET_TRANS_ID,MATCH_BET_ID,ACCOUNT_ID,COUNTRY_OF_RESIDENCE_NAME,PARENT_EVENT_ID,EVENT_ID,MATCH,EVENT_NAME,EVENT_DT,...,SETTLED_DATE,CANCELLED_DATE,SELECTION_NAME,PERSISTENCE_TYPE,BET_PRICE,PRICE_TAKEN,INPLAY_BET,BET_SIZE,PROFIT_LOSS,correct_profit_loss
19539,4748017907,5890112019,4748018137,1000819,United Kingdom,27327747,101092958,New Zealand v Sri Lanka,Match Odds,13/02/2015 22:00,...,14/02/2015 5:51,0,Sri Lanka,0,4.80,4.80,Y,77.133008,-77.133008,-77.133008
0,4755948997,5899638727,4755952732,1009306,UAE,27327749,101093076,India v Pakistan,Match Odds,15/02/2015 3:30:00 AM,...,15/02/2015 11:28:54 AM,0,India,0,1.61,1.61,Y,257.526603,-102.007575,-157.091228
1,4755948997,5899639259,4755953131,1009306,UAE,27327749,101093076,India v Pakistan,Match Odds,15/02/2015 3:30:00 AM,...,15/02/2015 11:28:54 AM,0,India,0,1.61,1.61,Y,386.289904,-153.014581,-235.636841
2,4755948997,5899635908,4755950405,1009306,UAE,27327749,101093076,India v Pakistan,Match Odds,15/02/2015 3:30:00 AM,...,15/02/2015 11:28:54 AM,0,India,0,1.61,1.61,Y,25.752660,-10.203204,-15.709123
3,4755948997,5899637522,4755951701,1009306,UAE,27327749,101093076,India v Pakistan,Match Odds,15/02/2015 3:30:00 AM,...,15/02/2015 11:28:54 AM,0,India,0,1.61,1.61,Y,321.908253,-127.510434,-196.364035


In [24]:
four_team_pre_dat.shape

(1202603, 25)

In [44]:
four_team_pre_dat_by_account = four_team_pre_dat.groupby('ACCOUNT_ID')
four_team_prefit_rate = four_team_pre_dat_by_account['correct_profit_loss'].sum() / four_team_pre_dat_by_account['BET_SIZE'].sum()
four_team_prefit_rate = four_team_prefit_rate.loc[submit.ACCOUNT_ID.tolist()]
four_team_prefit_rate = four_team_prefit_rate.fillna(four_team_prefit_rate.min())
four_team_prefit_rate = four_team_prefit_rate.reset_index()
four_team_prefit_rate.columns = ['Account_ID', 'Prediction']

In [45]:
four_team_prefit_rate.head()

,Account_ID,Prediction
0,1004240,0.002931
1,1009742,0.012451
2,1003908,0.001878
3,1020888,-0.470000
4,1010074,-0.140000


In [47]:
four_team_prefit_rate.to_csv('four_team_profit_rate_with_cancel.csv', index=False)

In [48]:
four_team_prefit_rate.isnull().any()

Account_ID    False
Prediction    False
dtype: bool

In [31]:
submit.shape

(7374, 2)

In [54]:
four_team_pre_dat = four_team_pre_dat.loc[(four_team_pre_dat.STATUS_ID == 'S').values]
four_team_pre_dat_by_account = four_team_pre_dat.groupby('ACCOUNT_ID')
four_team_prefit_rate = four_team_pre_dat_by_account['correct_profit_loss'].sum() / four_team_pre_dat_by_account['BET_SIZE'].sum()
four_team_prefit_rate = four_team_prefit_rate.loc[submit.ACCOUNT_ID.tolist()]
four_team_prefit_rate = four_team_prefit_rate.fillna(four_team_prefit_rate.min())
four_team_prefit_rate = four_team_prefit_rate.reset_index()
four_team_prefit_rate.columns = ['Account_ID', 'Prediction']

In [55]:
four_team_prefit_rate.shape

(7374, 2)

In [56]:
four_team_prefit_rate.to_csv('four_team_profit_rate_without_cancel.csv', index=False)

In [53]:
four_team_pre_dat.STATUS_ID.value_counts()

S    971679
dtype: int64

In [58]:
four_team_pre_dat = four_team_pre_dat.loc[(four_team_pre_dat.STATUS_ID == 'S').values]
four_team_pre_dat_by_account = four_team_pre_dat.groupby('ACCOUNT_ID')
four_team_prefit_rate = four_team_pre_dat_by_account['correct_profit_loss'].sum()
four_team_prefit_rate = four_team_prefit_rate.loc[submit.ACCOUNT_ID.tolist()]
four_team_prefit_rate = four_team_prefit_rate.fillna(four_team_prefit_rate.min())
four_team_prefit_rate = four_team_prefit_rate.reset_index()
four_team_prefit_rate.columns = ['Account_ID', 'Prediction']

In [59]:
four_team_prefit_rate.to_csv('four_team_profit_without_cancel.csv', index=False)

In [62]:
four_team_prefit_rate.shape

(7374, 2)

In [45]:
assert '3.1415926535'.isdecimal()

AssertionError: 

In [52]:
a = '3'

In [53]:
a.isdecimal()

True